# Importing Packages

In [1]:
import numpy as np
from scipy import linalg as splinalg
import matplotlib.pyplot as plt
from scipy import sparse as sp
import scipy.sparse.linalg as splin
from functools import reduce
import itertools
from scipy import linalg
from scipy.linalg import expm, logm
from scipy.special import comb
from itertools import combinations_with_replacement, product
from collections import Counter
import copy
from scipy.linalg import ishermitian
from scipy.linalg import eigh
import scipy

# Definitions

In [2]:
Z = sp.csc_matrix([[1, 0], [0, -1]])
X = sp.csc_matrix([[0, 1], [1, 0]])
Y = sp.csc_matrix([[0, -1j], [1j, 0]])
I = sp.csc_matrix([[1, 0], [0, 1]])

params={}
params['sites'] = 5
params['I'] = sp.eye(2)
params['X'] = X
params['Y'] = Y
params['Z'] = Z

In [3]:
print(params['Y'].toarray())

[[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]


In [4]:
print(Y.toarray())

[[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]


In [5]:
H = sp.csc_matrix((2**params['sites'], 2**params['sites']))
# H.toarray()

# Heisenberg Hamiltonian 1D

In [6]:
H = sp.csc_matrix((2**params['sites'], 2**params['sites']))
for i in range(params['sites']-1):
    H += reduce(sp.kron, (sp.eye(2**i), params['X'], params['X'], sp.eye(2**(params['sites']-2-i)))) + reduce(sp.kron, (sp.eye(2**i), params['Y'], params['Y'], sp.eye(2**(params['sites']-2-i)))) + reduce(sp.kron, (sp.eye(2**i), params['Z'], params['Z'], sp.eye(2**(params['sites']-2-i))))

H

<Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 84 stored elements and shape (32, 32)>

In [7]:
H.toarray()

array([[4.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 2.+0.j, 2.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 2.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       ...,
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 2.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 2.+0.j, 2.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 4.+0.j]])

In [8]:
# eigenvalues, eigenvectors = splin.eigsh(H, k=1, which='SA')
eigenvalues, eigenvectors = np.linalg.eigh(H.toarray())

In [9]:
eigenvalues[0]

np.float64(-7.711545013271978)

In [10]:
eigenvectors
print(eigenvectors[:,0])
ground_state = eigenvectors[:, 0]  

[ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  4.79886073e-02+0.j  0.00000000e+00+0.j -2.33021760e-01+0.j
  2.33021760e-01+0.j  1.11022302e-16+0.j  0.00000000e+00+0.j
  3.84446768e-01+0.j -6.65457136e-01+0.j  2.47693730e-17+0.j
  2.33021760e-01+0.j  4.11440961e-19+0.j  2.60847674e-17+0.j
  2.45571792e-17+0.j  0.00000000e+00+0.j -1.99413616e-01+0.j
  3.84446768e-01+0.j  2.21482895e-17+0.j -2.33021760e-01+0.j
  5.71321776e-17+0.j -1.51308573e-17+0.j  1.14786795e-17+0.j
  4.79886073e-02+0.j  4.02228408e-18+0.j  5.26082480e-17+0.j
  1.33026340e-17+0.j  4.44424914e-17+0.j -1.28858261e-17+0.j
 -4.88380365e-18+0.j  0.00000000e+00+0.j]


# Counting number of commuting local operators

In [40]:

def generate_heisenberg_hamiltonian(N, params):
    H = sp.csc_matrix((2**N, 2**N), dtype=complex)
    for i in range(N - 1):
        XX = reduce(sp.kron, (sp.eye(2**i), params['X'], params['X'], sp.eye(2**(N - 2 - i))))
        YY = reduce(sp.kron, (sp.eye(2**i), params['Y'], params['Y'], sp.eye(2**(N - 2 - i))))
        ZZ = reduce(sp.kron, (sp.eye(2**i), params['Z'], params['Z'], sp.eye(2**(N - 2 - i))))
        H += XX + YY + ZZ  
    return H

def generate_two_site_pauli_operators(N, params):
    operators = []
    Pauli_matrices = {'X': params['X'], 'Y': params['Y'], 'Z': params['Z']}
    
    for i in range(N - 1):
        j = i + 1 
        for pauli1 in Pauli_matrices.values():
            for pauli2 in Pauli_matrices.values():
                op = reduce(sp.kron, (sp.eye(2**i), pauli1, sp.eye(2**(j - i - 1)), pauli2, sp.eye(2**(N - j - 1))))
                operators.append(op)
    
    return operators

def count_commuting_operators(H, operators, tol=1e-8):
    num_commuting = 0
    non_commuting = 0
    # H_dense = H.toarray()  
    for op in operators:
        # op_dense = op.toarray()  
        # commutator = np.dot(H_dense, op_dense) - np.dot(op_dense, H_dense)  
        # if np.allclose(commutator, 0, atol=tol):  
        #     num_commuting += 1
        # else:
        #     non_commuting += 1
        
        commutator = (H@op) - (op@H)  
        if commutator.data.size == 0:
            num_commuting += 1
        else:
            non_commuting += 1
    
    return num_commuting, non_commuting

# Parameters
params = {
    'sites': 5,
    'I': sp.eye(2),
    'X': sp.csc_matrix([[0, 1], [1, 0]]),
    'Y': sp.csc_matrix([[0, -1j], [1j, 0]]),
    'Z': sp.csc_matrix([[1, 0], [0, -1]])
}

H = generate_heisenberg_hamiltonian(params['sites'], params)

# one_site_operators = generate_one_site_pauli_operators(params['sites'])
# num_commuting, non_commuting = count_commuting_operators(H, one_site_operators)

# two_site_operators = generate_two_site_pauli_operators(params['sites'], params)
# num_commuting, non_commuting = count_commuting_operators(H, two_site_operators)

# two_site_operators_all = generate_two_site_pauli_operators_all(params['sites'])
# num_commuting, non_commuting = count_commuting_operators(H, two_site_operators_all)

all_operators = generate_one_and_two_site_pauli_operators(params['sites'], params)
num_commuting, non_commuting = count_commuting_operators(H, all_operators)

print(f"Number of commuting operators with the Hamiltonian: {num_commuting}")
print(f"Number of non-commuting operators with the Hamiltonian: {non_commuting}")

Number of commuting operators with the Hamiltonian: 0
Number of non-commuting operators with the Hamiltonian: 51


# Correlation matrix method

## One-site local operators

In [11]:
def generate_one_site_pauli_operators(N):
    one_site_ops = []
    for i in range(N - 1): 
        X_i = reduce(sp.kron, (sp.eye(2**i), params['X'], sp.eye(2**(N - i - 1))))
        Y_i = reduce(sp.kron, (sp.eye(2**i), params['Y'], sp.eye(2**(N - i - 1))))
        Z_i = reduce(sp.kron, (sp.eye(2**i), params['Z'], sp.eye(2**(N - i - 1))))
        one_site_ops.extend([X_i, Y_i, Z_i])
    return one_site_ops

one_site_operators = generate_one_site_pauli_operators(params['sites'])
# print(two_site_operators)

num_ops = len(one_site_operators)
# covariance_matrix = sp.csc_matrix((num_ops, num_ops), dtype=complex)
correlation_matrix = np.zeros((num_ops, num_ops), dtype=complex)

for i in range(num_ops):
    for j in range(num_ops):
        V_i = one_site_operators[i].toarray() 
        V_j = one_site_operators[j].toarray() 
        term1 = np.vdot(ground_state, np.dot(V_i, np.dot(V_j, ground_state)))
        term2 = np.vdot(ground_state, np.dot(V_i, ground_state)) * np.vdot(ground_state, np.dot(V_j, ground_state))
        # print(term1-term2)
        correlation_matrix[i, j] = term1 - term2  
        
# eigenvalues, eigenvectors = splin.eigsh(covariance_matrix)
eigenvalues1, eigenvectors1 = np.linalg.eigh(correlation_matrix)

## Eigenvectors with zero eigenvalue (local operators that commute with the ground state)
zero_eigenvectors1 = eigenvectors1[:, np.isclose(eigenvalues1, 0, atol=1e-8)]

print("Eigenvectors with zero eigenvalue of the covariance matrix (one-site Pauli operators):")
for i in range(zero_eigenvectors1.shape[1]):
    print(f"Eigenvector {i + 1}:")
    print(zero_eigenvectors1[:, i])

# # Reconstruct the effective Hamiltonian in the subspace of zero eigenvectors
# # Use the zero eigenvectors to construct an effective Hamiltonian
effective_hamiltonian_correl = np.zeros((2**params['sites'], 2**params['sites']), dtype=complex)
for i in range(zero_eigenvectors1.shape[1]):
    coeff = zero_eigenvectors1[:, i]
    V_eff = sum(coeff[j] * one_site_operators[j].toarray() for j in range(num_ops))
    effective_hamiltonian_correl += V_eff

print("\nEffective Hamiltonian in the subspace of zero eigenvectors:")
print(effective_hamiltonian_correl)

Eigenvectors with zero eigenvalue of the covariance matrix (one-site Pauli operators):

Effective Hamiltonian in the subspace of zero eigenvectors:
[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 ...
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]


In [12]:
correlation_matrix.shape

(12, 12)

In [13]:
num_ops

12

In [14]:
eigenvalues1

array([3.61651470e-04, 8.82677257e-02, 9.24959029e-02, 1.66225626e-01,
       4.81294818e-01, 4.92669913e-01, 1.05712700e+00, 1.21784050e+00,
       1.34853035e+00, 1.60679831e+00, 2.20096764e+00, 2.49228351e+00])

In [15]:
(correlation_matrix==correlation_matrix.T.conj()).all()

np.False_

In [16]:
ishermitian(correlation_matrix)

False

## Two-site local operators

### For nearest neighbor interactions 

In [17]:
def generate_two_site_pauli_operators(N):
    two_site_ops = []
    Pauli_matrices = {'X': np.array([[0, 1], [1, 0]]),
                      'Y': np.array([[0, -1j], [1j, 0]]),
                      'Z': np.array([[1, 0], [0, -1]])}
    for i in range(N - 1):
        j = i + 1  
        for pauli1 in Pauli_matrices.values():
            for pauli2 in Pauli_matrices.values():
                op = reduce(sp.kron, (sp.eye(2**i), pauli1, sp.eye(2**(j - i - 1)), pauli2, sp.eye(2**(N - j - 1))))
                two_site_ops.append(op)

    
    return two_site_ops

two_site_operators = generate_two_site_pauli_operators(params['sites'])
# print(two_site_operators)

num_ops = len(two_site_operators)
# covariance_matrix = sp.csc_matrix((num_ops, num_ops), dtype=complex)
correlation_matrix = np.zeros((num_ops, num_ops), dtype=complex)

for i in range(num_ops):
    for j in range(num_ops):
        V_i = two_site_operators[i].toarray() 
        V_j = two_site_operators[j].toarray() 
        term1 = np.vdot(ground_state, np.dot(V_i, np.dot(V_j, ground_state)))
        term2 = np.vdot(ground_state, np.dot(V_i, ground_state)) * np.vdot(ground_state, np.dot(V_j, ground_state))
        # print(term1-term2)
        correlation_matrix[i, j] = term1 - term2  
        
# eigenvalues, eigenvectors = splin.eigsh(covariance_matrix)
eigenvalues1, eigenvectors1 = np.linalg.eigh(correlation_matrix)

## Eigenvectors with zero eigenvalue (local operators that commute with the ground state)
zero_eigenvectors1 = eigenvectors1[:, np.isclose(eigenvalues1, 0, atol=1e-15)]

print("Eigenvectors with zero eigenvalue of the correlation matrix (two-site Pauli operators):")
for i in range(zero_eigenvectors1.shape[1]):
    print(f"Eigenvector {i + 1}:")
    print(zero_eigenvectors1[:, i])

# # Reconstruct the effective Hamiltonian in the subspace of zero eigenvectors
# # Use the zero eigenvectors to construct an effective Hamiltonian
effective_hamiltonian_correl = np.zeros((2**params['sites'], 2**params['sites']), dtype=complex)
for i in range(zero_eigenvectors1.shape[1]):
    coeff = zero_eigenvectors1[:, i]
    V_eff = sum(coeff[j] * two_site_operators[j].toarray() for j in range(num_ops))
    effective_hamiltonian_correl += V_eff

print("\nEffective Hamiltonian in the subspace of zero eigenvectors:")
print(effective_hamiltonian_correl)

Eigenvectors with zero eigenvalue of the correlation matrix (two-site Pauli operators):
Eigenvector 1:
[ 7.22900102e-01+0.00000000e+00j -3.25110682e-05+2.31363879e-01j
  3.13838389e-06+3.14565611e-06j -1.82561776e-05+4.46137962e-01j
  4.53982612e-02-5.07672458e-05j -3.14565611e-06+3.13838389e-06j
  3.57001824e-06+1.75010455e-07j -1.75010453e-07+3.57001824e-06j
  7.51250568e-02-5.28431333e-06j  1.13232424e-01-8.16941360e-06j
 -5.18249732e-06-8.60470977e-02j -2.41053247e-06+1.91790189e-06j
  1.78753672e-06+1.77068280e-02j  1.81572694e-01-1.15643742e-05j
 -1.91790189e-06-2.41053247e-06j -1.02702374e-06+2.70657714e-07j
 -2.70657715e-07-1.02702374e-06j  2.11634555e-01-1.38872149e-05j
  4.91598258e-02-1.05368288e-05j  1.59155388e-06-8.60220097e-02j
  4.60585814e-07-7.66983392e-07j  8.56158792e-06+1.77319160e-02j
  1.17449919e-01-3.83687040e-07j  7.66983394e-07+4.60585812e-07j
  1.17984080e-06-2.99626168e-06j  2.99626168e-06+1.17984080e-06j
  1.90728765e-02-1.43993699e-06j -1.60469690e-01+1.0

In [18]:
correlation_matrix.shape

(36, 36)

In [19]:
num_ops

36

In [20]:
eigenvalues1
(correlation_matrix==correlation_matrix.T.conj()).all()

np.False_

In [21]:
ishermitian(correlation_matrix)

False

In [22]:
eigenvalues1

array([-1.75088628e-15, -2.66153361e-16, -2.40143234e-16, -1.45721579e-16,
       -4.53100981e-17,  1.01364724e-16,  1.50417705e-16,  2.99284112e-16,
        3.18781651e-16,  2.78582527e-03,  4.82560278e-03,  1.58348762e-02,
        3.11355694e-02,  4.87244865e-02,  1.02669308e-01,  1.30924577e-01,
        2.00507800e-01,  2.23159386e-01,  2.66481807e-01,  2.78707546e-01,
        2.92880594e-01,  3.02388820e-01,  4.52816376e-01,  6.43675122e-01,
        1.03043779e+00,  1.31115647e+00,  1.55840560e+00,  1.95092862e+00,
        2.10780764e+00,  2.40216827e+00,  2.49080563e+00,  2.78322955e+00,
        2.81760033e+00,  2.85657598e+00,  3.15699780e+00,  3.37489222e+00])

In [23]:
num_ops

36

### For all types of two-site interactions

In [24]:
def generate_two_site_pauli_operators_all(N):
    two_site_ops = []
    Pauli_matrices = {'X': np.array([[0, 1], [1, 0]]),
                      'Y': np.array([[0, -1j], [1j, 0]]),
                      'Z': np.array([[1, 0], [0, -1]])}
    # For all possible combinations    
    for i in range(N - 1):
        for j in range(i+1, N):
            for pauli1 in Pauli_matrices.values():
                for pauli2 in Pauli_matrices.values():
                    op = reduce(sp.kron, (sp.eye(2**i), pauli1, sp.eye(2**(j - i - 1)), pauli2, sp.eye(2**(N - j - 1))))
                    two_site_ops.append(op)
    return two_site_ops

two_site_operators = generate_two_site_pauli_operators_all(params['sites'])
# print(two_site_operators)

num_ops = len(two_site_operators)
# covariance_matrix = sp.csc_matrix((num_ops, num_ops), dtype=complex)
correlation_matrix = np.zeros((num_ops, num_ops), dtype=complex)

for i in range(num_ops):
    for j in range(num_ops):
        V_i = two_site_operators[i].toarray() 
        V_j = two_site_operators[j].toarray() 
        term1 = np.vdot(ground_state, np.dot(V_i, np.dot(V_j, ground_state)))
        term2 = np.vdot(ground_state, np.dot(V_i, ground_state)) * np.vdot(ground_state, np.dot(V_j, ground_state))
        # print(term1-term2)
        correlation_matrix[i, j] = term1 - term2  
        
# eigenvalues, eigenvectors = splin.eigsh(covariance_matrix)
eigenvalues1, eigenvectors1 = np.linalg.eigh(correlation_matrix)

## Eigenvectors with zero eigenvalue (local operators that commute with the ground state)
zero_eigenvectors1 = eigenvectors1[:, np.isclose(eigenvalues1, 0, atol=1e-15)]

print("Eigenvectors with zero eigenvalue of the correlation matrix (two-site Pauli operators):")
for i in range(zero_eigenvectors1.shape[1]):
    print(f"Eigenvector {i + 1}:")
    print(zero_eigenvectors1[:, i])

# # Reconstruct the effective Hamiltonian in the subspace of zero eigenvectors
# # Use the zero eigenvectors to construct an effective Hamiltonian
effective_hamiltonian_correl = np.zeros((2**params['sites'], 2**params['sites']), dtype=complex)
for i in range(zero_eigenvectors1.shape[1]):
    coeff = zero_eigenvectors1[:, i]
    V_eff = sum(coeff[j] * two_site_operators[j].toarray() for j in range(num_ops))
    effective_hamiltonian_correl += V_eff

print("\nEffective Hamiltonian in the subspace of zero eigenvectors:")
print(effective_hamiltonian_correl)

Eigenvectors with zero eigenvalue of the correlation matrix (two-site Pauli operators):
Eigenvector 1:
[ 3.77220793e-16+0.00000000e+00j  2.08070362e-01+2.07832007e-01j
 -2.66342423e-02-7.38772696e-02j  6.72952937e-02-4.15154037e-02j
 -1.61746853e-01+8.25896742e-02j -3.45898596e-01-2.90369325e-02j
 -3.17709945e-02-8.77019196e-02j -1.37271014e-01-5.62380762e-02j
  3.83664474e-02+1.01204116e-02j  1.30892305e-01-9.11734339e-03j
 -4.12155019e-03+1.16395558e-01j -1.11447833e-02-1.10535954e-01j
  1.48378773e-01-1.22983039e-01j  1.80498214e-02-4.77710102e-02j
 -2.15513210e-01-1.58311627e-02j  1.00752047e-01-9.60922182e-03j
  3.52574739e-02+9.97318179e-03j  2.81559632e-02+6.30372672e-02j
 -9.62172587e-02+3.54767552e-05j  5.43237139e-02+4.25185372e-02j
 -8.58651688e-02-8.94970241e-03j  1.27844138e-01-1.29903766e-01j
 -3.93718447e-02+5.60620169e-02j -4.23579215e-02-8.09508583e-02j
  7.41745527e-02+5.02345005e-02j  3.38859389e-02-2.32956853e-02j
  2.89319108e-01+2.53168816e-02j -5.37389657e-02+1.7

In [25]:
correlation_matrix.shape

(90, 90)

In [26]:
num_ops

90

In [27]:
eigenvalues1
(correlation_matrix==correlation_matrix.T.conj()).all()

np.False_

In [28]:
ishermitian(correlation_matrix)

False

In [29]:
eigenvalues1

array([-2.00956834e-15, -9.95548469e-16, -8.32827037e-16, -8.30834136e-16,
       -7.83748619e-16, -7.20305397e-16, -7.19068853e-16, -6.42045554e-16,
       -5.86122451e-16, -5.47752044e-16, -5.27482836e-16, -5.10319171e-16,
       -4.59769688e-16, -4.45233059e-16, -3.85044266e-16, -3.73468433e-16,
       -3.11343465e-16, -3.01080692e-16, -2.54556357e-16, -2.53980993e-16,
       -2.06346011e-16, -1.50981707e-16, -1.49304097e-16, -1.46374359e-16,
       -1.41512378e-16, -1.18398423e-16, -7.84022112e-17, -4.16430214e-17,
       -2.77181586e-17, -8.19751391e-18,  1.51076621e-17,  3.93530043e-17,
        6.17339419e-17,  8.52627282e-17,  1.24561595e-16,  1.28879874e-16,
        1.57480826e-16,  1.90885836e-16,  2.52510352e-16,  2.75434306e-16,
        2.77694894e-16,  3.07474740e-16,  3.37872095e-16,  3.41839466e-16,
        4.04999347e-16,  4.74842462e-16,  4.84779608e-16,  5.20224068e-16,
        5.49474305e-16,  5.64262620e-16,  6.08364309e-16,  7.13158023e-16,
        7.32672809e-16,  

In [30]:
num_ops

90

# One-site + two-site local interactions

In [31]:
def generate_one_and_two_site_pauli_operators(N, params):
    """
    Returns: one-site and two-site (with only nearest neighbor interactions) local operators
    """
    operators = []
    Pauli_matrices = {'X': params['X'], 'Y': params['Y'], 'Z': params['Z']}

    ## One-site local operators
    for i in range(N):
        for pauli in Pauli_matrices.values():
            op = reduce(sp.kron, (sp.eye(2**i), pauli, sp.eye(2**(N - i - 1))))
            operators.append(op)
    
    ## Two-site local operators - nearest neighbor interactions
    for i in range(N - 1):
        j = i + 1  
        for pauli1 in Pauli_matrices.values():
            for pauli2 in Pauli_matrices.values():
                op = reduce(sp.kron, (sp.eye(2**i), pauli1, sp.eye(2**(j - i - 1)), pauli2, sp.eye(2**(N - j - 1))))
                operators.append(op)
    
    return operators

all_operators = generate_one_and_two_site_pauli_operators(params['sites'], params)

In [32]:
num_ops = len(all_operators)
# covariance_matrix = sp.csc_matrix((num_ops, num_ops), dtype=complex)
correlation_matrix = np.zeros((num_ops, num_ops), dtype=complex)

for i in range(num_ops):
    for j in range(num_ops):
        V_i = all_operators[i].toarray() 
        V_j = all_operators[j].toarray() 
        term1 = np.vdot(ground_state, np.dot(V_i, np.dot(V_j, ground_state)))
        term2 = np.vdot(ground_state, np.dot(V_i, ground_state)) * np.vdot(ground_state, np.dot(V_j, ground_state))
        # print(term1-term2)
        correlation_matrix[i, j] = term1 - term2  
        
# eigenvalues, eigenvectors = splin.eigsh(covariance_matrix)
eigenvalues1, eigenvectors1 = np.linalg.eigh(correlation_matrix)

## Eigenvectors with zero eigenvalue (local operators that commute with the ground state)
zero_eigenvectors1 = eigenvectors1[:, np.isclose(eigenvalues1, 0, atol=1e-15)]

print("Eigenvectors with zero eigenvalue of the correlation matrix:")
for i in range(zero_eigenvectors1.shape[1]):
    print(f"Eigenvector {i + 1}:")
    print(zero_eigenvectors1[:, i])

# # Reconstruct the effective Hamiltonian in the subspace of zero eigenvectors
# # Use the zero eigenvectors to construct an effective Hamiltonian
effective_hamiltonian_correl = np.zeros((2**params['sites'], 2**params['sites']), dtype=complex)
for i in range(zero_eigenvectors1.shape[1]):
    coeff = zero_eigenvectors1[:, i]
    V_eff = sum(coeff[j] * all_operators[j].toarray() for j in range(num_ops))
    effective_hamiltonian_correl += V_eff

print("\nEffective Hamiltonian in the subspace of zero eigenvectors:")
print(effective_hamiltonian_correl)

Eigenvectors with zero eigenvalue of the correlation matrix:
Eigenvector 1:
[ 0.        +0.00000000e+00j -0.03971152+4.39060654e-02j
  0.33627885+1.19775662e-01j  0.22764064+1.03408773e-01j
  0.14218134-1.71867166e-01j -0.04991987-3.01809073e-04j
  0.14564844+2.68066627e-02j  0.10550712-1.25652537e-01j
  0.02381975+4.15819601e-02j -0.06336891-4.69673428e-02j
 -0.045578  +2.23303358e-02j -0.08754036+7.33949108e-02j
  0.00045913-1.07464390e-01j  0.08899217+7.58551221e-02j
 -0.14108917+1.00842482e-01j  0.01959766-4.61753111e-02j
  0.0465394 -4.62608616e-01j -0.13066086-2.81367318e-03j
 -0.03016727+6.25906492e-02j  0.41961562-2.98031808e-02j
 -0.10982672+5.08367944e-02j  0.03244666+4.89446623e-02j
  0.06732322-1.34663039e-01j -0.05267966+6.22985084e-02j
  0.0198744 +2.76103666e-02j -0.01806345-6.61643554e-02j
  0.04480667+6.26828660e-02j -0.00528783+3.11816793e-02j
  0.05485708+4.25908194e-03j -0.04760363+2.50788844e-02j
  0.01333503+9.42611805e-04j -0.02146734+3.20312670e-02j
  0.06976798